In [ ]:
import pandas as pd
import numpy as np
from matplotlib.cm import ScalarMappable
from matplotlib.colors import to_hex, Normalize, ListedColormap
from reframed import load_cbmodel
from reframed.alpha.MARGE2 import marge
import seaborn as sns

### Select strain

In [ ]:
strain = '322b' # L kefiranofaciens
#strain = '230a' # L mesenteroides
#strain = '261'  # L lacis

### Load gene expression data

In [ ]:
df = pd.read_excel("../transcriptomics/deseq2_results.xlsx", sheet_name=strain)
df["gene"] = df["Unnamed: 0"].apply(lambda x:  "_".join(x.split("_")[1:]))
df.drop(columns=["Unnamed: 0", "baseMean", "lfcSE", "stat", "pvalue", "color", "names"], inplace=True)

In [ ]:
rel_expr = {x["gene"]: 2**x["log2FoldChange"] for _, x in df.iterrows() if x["padj"] < 0.05}

### Load model

In [ ]:
model = load_cbmodel(f"../models/{strain}_03182016.xml", flavor="cobra")

### Estimate co-culture fluxes 

Based on the MARGE approach (Metabolic Analysis with Relative Gene Expression)

Uses mono-culture as the reference state and integrates relative gene expression levels between co-cultures and mono-cultures

In [ ]:
v1, v2, sol1, sol2 = marge(model, rel_expr, transformed=False, 
                           growth_frac_a=0.9, growth_frac_b=0.3, activation=0.1)

In [ ]:
entries = []
for rxn in model.reactions:
    f1 = abs(v1[rxn])
    f2 = abs(v2[rxn])
    if f1 > 0:
        if f2 > 0:
            fc = np.log2(f2/f1)
        else:
            fc = -10
    else:
        if f2 > 0:
            fc = 10
        else:
            fc = 0
    entries.append((rxn, v1[rxn], v2[rxn], fc))
fluxes = pd.DataFrame(entries, columns=["bigg", "v1", "v2", "fc"])

### Map to KEGG reactions using MetaNetX

In [ ]:
mnx = pd.read_csv("../misc_data/reac_xref.tsv", sep="\t", comment='#', 
                  usecols=[0,1], header=None, names=["ext", "int"])

In [ ]:
bigg = mnx[mnx["ext"].str.startswith("bigg:R_")].copy()
bigg["bigg"] = bigg["ext"].apply(lambda x: x[5:])
kegg = mnx[mnx["ext"].str.startswith("kegg:")].copy()
kegg["kegg"] = kegg["ext"].apply(lambda x: x[5:])
bigg2kegg = pd.merge(bigg, kegg, on="int")
del bigg2kegg["ext_x"]
del bigg2kegg["ext_y"]
del bigg2kegg["int"]

In [ ]:
fluxes = pd.merge(fluxes, bigg2kegg, on="bigg")

In [ ]:
fluxes = fluxes.groupby("kegg").agg({"fc": np.mean})

## Draw in iPATH

In [ ]:
cmap = ListedColormap(sns.color_palette("RdYlBu_r", n_colors=100))
mapper = ScalarMappable(norm=Normalize(-3, 3), cmap=cmap)
fluxes["color"] = fluxes["fc"].apply(lambda x: to_hex(mapper.to_rgba(x)))
fluxes["width"] = "W20"
fluxes[["color", "width"]].to_csv(f"../iPATH/{strain}.tsv", sep="\t", header=False)

The resulting TSV files can be uploaded in https://pathways.embl.de/ipath3.cgi?map=metabolic to generate the flux maps.